In [18]:
from common_nba_api_functions import rate_limited_request
from common_db_functions import get_players_from_db
players = get_players_from_db()
player_ids = [player.player_id for player in players]
season = "2023-24"

In [19]:
from nba_api.stats.endpoints import playergamelog

games_by_player = []
for player_id in player_ids[:1]:
    gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2022-23')
    games_df = gamelog.get_data_frames()[0]
    games_by_player.append(games_df)
print(games_df.columns)

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')


In [21]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from db_config import get_session
from models import GameStats, Player
from common_nba_api_functions import rate_limited_request
session = get_session()


def fetch_and_store_player_game_stats(player_id):

    # Fetch player game logs
    gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
    games_df = gamelog.get_data_frames()[0]
    for index, game in games_df.iterrows():
        # Ensure player exists in the DB, necessary to link stats
        db_player = session.query(Player).filter(Player.player_id == player_id).first()
        if not db_player:
            continue  # Or handle adding a new player if this case is possible
        
        # Check if the game stats already exist to avoid duplicates
        existing_game_stat = session.query(GameStats).filter(GameStats.game_id == game['Game_ID'], GameStats.player_id == db_player.id).first()
        
        if existing_game_stat:
            # Update existing record with new data
            existing_game_stat.points = game['PTS']
            existing_game_stat.assists = game['AST']
            existing_game_stat.rebounds = game['REB']
            existing_game_stat.steals = game['STL']
            existing_game_stat.blocks = game['BLK']
            existing_game_stat.turnovers = game['TOV']
            existing_game_stat.fouls = game['PF']
            existing_game_stat.minutes = game['MIN']
            existing_game_stat.fg_made = game['FGM']
            existing_game_stat.fg_attempts = game['FGA']
            existing_game_stat.fg_percentage = game['FG_PCT']
            existing_game_stat.fg3_made = game['FG3M']
            existing_game_stat.fg3_attempts = game['FG3A']
            existing_game_stat.fg3_percentage = game['FG3_PCT']
            existing_game_stat.ft_made = game['FTM']
            existing_game_stat.ft_attempts = game['FTA']
            existing_game_stat.ft_percentage = game['FT_PCT']
            existing_game_stat.plus_minus = game['PLUS_MINUS']
        else:
            # Create and add new game stats record if it doesn't exist
            new_game_stat = GameStats(
                player_id=db_player.id,
                game_id=game['Game_ID'],
                points=game['PTS'],
                assists=game['AST'],
                rebounds=game['REB'],
                steals=game['STL'],
                blocks=game['BLK'],
                turnovers=game['TOV'],
                fouls=game['PF'],
                minutes=game['MIN'],
                fg_made=game['FGM'],
                fg_attempts=game['FGA'],
                fg_percentage=game['FG_PCT'],
                fg3_made=game['FG3M'],
                fg3_attempts=game['FG3A'],
                fg3_percentage=game['FG3_PCT'],
                ft_made=game['FTM'],
                ft_attempts=game['FTA'],
                ft_percentage=game['FT_PCT'],
                plus_minus=game['PLUS_MINUS']
            )
            session.add(new_game_stat)
    
    # Commit changes to the database
    session.commit()

for player_id in player_ids:
    rate_limited_request(2)
    fetch_and_store_player_game_stats(player_id)

    # Close session
    session.close()